In [3]:
from langchain_ollama.llms import OllamaLLM
from langchain_core.runnables import RunnableSequence
from langchain.output_parsers.pydantic import PydanticOutputParser
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate

from pydantic import BaseModel, Field

In [15]:
class SQLModel(BaseModel):
    sql: str = Field(description="Raw ready to use sql command with rollback")
    rollback_sql: str = Field(description="The rollback sql script incase of issue, (optional)")
    explanation: str = Field(description="Detailed description of the sql")

In [16]:
output = PydanticOutputParser(pydantic_object=SQLModel)

In [17]:
prompt = ChatPromptTemplate([
    ("system", "You are Natural Language to SQL Converter"),
    ("human", "I don't know sql, convert my natural language: {nl} to sql"),
    ("system", "Format the output in this format: {out_format}"),
])

In [18]:
llm = OllamaLLM(model="llama3:8b")

In [19]:
chat = RunnableSequence(prompt, llm, output)

In [20]:
output.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"sql": {"description": "Raw ready to use sql command with rollback", "title": "Sql", "type": "string"}, "rollback_sql": {"description": "The rollback sql script incase of issue, (optional)", "title": "Rollback Sql", "type": "string"}, "explanation": {"description": "Detailed description of the sql", "title": "Explanation", "type": "string"}}, "required": ["sql", "rollback_sql", "explanation"]}\n```'

In [21]:
chat.invoke({
    "nl": "show all staffs in pex who are dumb under age of 45",
    "out_format": output.get_format_instructions()
})

SQLModel(sql="SELECT * FROM staff WHERE is_dumb = 'true' AND age < 45;", rollback_sql='', explanation="This query selects all staff members from the 'staff' table where they are considered dumb (is_dumb column) and their age is less than 45.")

In [23]:
chat.invoke({
    "nl": "insert all staffs in pec who are dumb under age of 45",
    "out_format": output.get_format_instructions()
})

SQLModel(sql='INSERT INTO Staff (Name, Age, IQ) SELECT s.Name, s.Age, s.IQ FROM Staff s WHERE s.Age < 45 AND s.IQ <= 80;', rollback_sql='ROLLBACK;', explanation="This SQL command inserts all staffs in PEC who are considered 'dumb' (IQ score of 80 or below) and under the age of 45. The IQ scores are based on internal assessments and may not be accurate.")